In [14]:
import haskellian.either as E
from haskellian.either import IsLeft
from pydantic import BaseModel, Field
from quicktype_ts import pydantic2typescript

In [23]:
from typing import Generic, TypeVar, Literal, Callable, Any
from abc import ABC, abstractmethod
from dataclasses import dataclass
A = TypeVar('A')
L = TypeVar('L')
R = TypeVar('R')
L2 = TypeVar('L2')
R2 = TypeVar('R2')

class EitherBase(ABC, Generic[L, R]):

  @abstractmethod
  def match(self, on_left: Callable[[L], A], on_right: Callable[[R], A]) -> A:
    """Unwrap an `Either` by matching both branches"""

  @abstractmethod
  def unsafe(self) -> R:
    """Unwraps the value or throws an `IsLeft` exception
    
    (`IsLeft.value` will contain the wrapped value)"""
    

  def bind(self, f: 'Callable[[R], Either[L, R2]]') -> 'Either[L, R2]':
    return self.match(lambda x: Left(x), f)

  def fmap(self, f: Callable[[R], R2]) -> 'Either[L, R2]':
    return self.match(lambda x: Left(x), lambda x: Right(f(x)))

  def mapl(self, f: Callable[[L], L2]) -> 'Either[L2, R]':
    """Map the left branch"""
    return self.match(lambda x: Left(f(x)), lambda x: Right(x))

  __or__ = fmap
  """Alias of `fmap`"""
  
  __and__ = bind
  """Alias of `bind`"""
  
  def match_(self, on_left: Callable[[], A], on_right: Callable[[], A]) -> A:
    """Like `match`, but handlers don't get the wrapped value"""
    return self.match(lambda _: on_left(), lambda _: on_right())

@dataclass
class Left(EitherBase[L, Any], Generic[L]):
  value: L
  tag: Literal['left'] = 'left'

  def match(self, on_left: Callable[[L], A], on_right: Callable[[R], A]) -> A:
    return on_left(self.value)
  
  def unsafe(self) -> R:
    raise IsLeft(self.value)

@dataclass
class Right(EitherBase[Any, R], Generic[R]):
  value: R
  tag: Literal['right'] = 'right'

  def match(self, on_left: Callable[[L], A], on_right: Callable[[R], A]) -> A:
    return on_right(self.value)
  
  def unsafe(self) -> R:
    return self.value

Either = Left[L] | Right[R]

Left(2)

TypeError: Can't instantiate abstract class Left with abstract method match

In [20]:
import json
class Model(BaseModel):
  x: Either[int, str] = Field(discriminator='tag')


json.dumps(Model.model_json_schema())

'{"$defs": {"Left_int_": {"properties": {"left": {"title": "Left", "type": "integer"}, "tag": {"const": "left", "default": "left", "title": "Tag"}}, "required": ["left"], "title": "Left", "type": "object"}, "Right_str_": {"properties": {"right": {"title": "Right", "type": "string"}, "tag": {"const": "right", "default": "right", "title": "Tag"}}, "required": ["right"], "title": "Right", "type": "object"}}, "properties": {"x": {"discriminator": {"mapping": {"left": "#/$defs/Left_int_", "right": "#/$defs/Right_str_"}, "propertyName": "tag"}, "oneOf": [{"$ref": "#/$defs/Left_int_"}, {"$ref": "#/$defs/Right_str_"}], "title": "X"}}, "required": ["x"], "title": "Model", "type": "object"}'